Imports

In [2]:
import numpy as np
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
from Network import Network
from MaxPooling import MaxPooling
from FullyConnected import FullyConnected
from Convolution import Convolution
from CrossEntropyLossFunction import CrossEntropyLossFunction
from Sigmoid import Sigmoid
from Softmax import Softmax
from Tanh import Tanh
from Flatten import Flatten
from ReLU import ReLU

Extract Data

In [4]:
images = []
labels = []

for folder in os.listdir('asl_dataset'):
    folder_dir = os.path.join('asl_dataset', folder)

    for image in os.listdir(folder_dir):
        img_path = os.path.join(folder_dir, image)

        data_sample = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        data_sample = cv2.resize(data_sample, (64, 64))
        data_sample = data_sample / 255.0

        images.append(data_sample)
        labels.append(folder)

Preprocess the Data

In [5]:
df = pd.DataFrame(labels)
df = pd.get_dummies(df)

In [6]:
x_data = np.array(images).reshape(-1, 1, 64, 64)
y_data = df.values

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

Create the Model

In [13]:
loss_function = CrossEntropyLossFunction()

network_layers = [
    Convolution(input_shape=(1, 64, 64), output_depth=16, kernel_size=3),
    ReLU(),
    MaxPooling(pool_size=2, stride=2),

    Convolution(input_shape=(16, 31, 31), output_depth=32, kernel_size=3),
    ReLU(),
    MaxPooling(pool_size=2, stride=2),

    Convolution(input_shape=(32, 14, 14), output_depth=64, kernel_size=3),
    ReLU(),
    MaxPooling(pool_size=2, stride=2),

    Flatten(),

    FullyConnected(input_size=64 * 6 * 6, output_size=256),
    ReLU(),

    FullyConnected(input_size=256, output_size=y_data.shape[1]),
    Softmax()
]

In [14]:
ASL_model = Network(network_layers, loss_function)

In [ ]:
ASL_model.train(x_train, y_train, 20, 0.001)

Save the Model

In [ ]:
import pickle

filename = 'ASL_trained_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(ASL_model, file)